In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import pickle

## Analysis for DE-EN

In [10]:
df_all = pd.read_pickle("/home/nunomg/mt-hallucinations/HALO/fairseq/data-bin/wmt18_de-en_heldout/checkpoint_best/dataframes/heldoutwstats_w_bicleaner.pkl")
df = pd.read_pickle("/home/nunomg/mt-hallucinations/HALO/fairseq/data-bin/wmt18_de-en_heldout/checkpoint_best/dataframes/heldout_lowcomet_w_bicleaner.pkl")

In [11]:
with open("/home/nunomg/mt-hallucinations/HALO/fairseq/data-bin/wmt18_de-en_heldout/checkpoint_best/mc-dropout-scores-lowcomet_w_bicleaner/agg_scores.levdist_w_bicleaner.pkl", "rb") as f:
    levdist_lowcomet = pickle.load(f)
with open("/home/nunomg/mt-hallucinations/HALO/fairseq/data-bin/wmt18_de-en_heldout/checkpoint_best/mc-dropout-scores-lowcomet_w_bicleaner/agg_ensembleprobs_scores_w_bicleaner.pkl", "rb") as f:
    logprobs_lowcomet = pickle.load(f)

In [12]:
df["var_probs"] = list(np.array(logprobs_lowcomet[0]) ** 2)
df["avg_div_levdist"] = list(np.array(levdist_lowcomet[0]))

### Add Bicleaner scores

In [11]:
bicleaner_scores = []
for idx in df["idx"].values:
    bicleaner_scores.append(float(df_all.loc[df_all["idx"]==idx].bicleaner_score.values[0]))
len(bicleaner_scores)

14108

In [4]:
df["bicleaner_score"] = bicleaner_scores

### Define Thresholds

In [13]:
epsilon = 1
score_threshold = df_all.sort_values(by="score")[:round(epsilon/100 * len(df_all))]["score"].values[-1]
df_score_filter = df.loc[df["score"] <= score_threshold]

sim_div_threshold = df.sort_values(by="avg_div_levdist", ascending=False)[:len(df_score_filter)]["avg_div_levdist"].values[-1]
df_divergence_filter = df.sort_values(by="avg_div_levdist")[:len(df_score_filter)]

var_probs_threshold = df.sort_values(by="var_probs", ascending=False)[:len(df_score_filter)]["var_probs"].values[-1]
df_var_probs = df.sort_values(by="var_probs")[:len(df_score_filter)]

In [14]:
df_lss_lsd = df.loc[(df["score"]<= score_threshold) | (df["avg_div_levdist"] >= sim_div_threshold)]
len(df_lss_lsd)

2761

In [15]:
df_lss_lvp = df.loc[(df["score"]<= score_threshold) | (df["var_probs"] >= var_probs_threshold)]
len(df_lss_lvp)

3359

In [16]:
df_lsd_lvp = df.loc[(df["avg_div_levdist"] >= sim_div_threshold) | (df["var_probs"] >= var_probs_threshold)]
len(df_lsd_lvp)

3344

### Other criteria

In [17]:
len(df.loc[(df["f1_bpe"]==1) & (df["score"] > score_threshold) & (df["avg_div_levdist"] < sim_div_threshold)])

35

In [54]:
len(df.loc[(df["f1_bpe"]==1) & (df["score"] > score_threshold) & (df["var_probs"] < var_probs_threshold)])

39

In [55]:
len(df.loc[(df["f1_bpe"]==1) & (df["avg_div_levdist"] < sim_div_threshold) & (df["var_probs"] < var_probs_threshold)])

4

In [66]:
mt = df.loc[(df["f1_bpe"]==1) & (df["score"] > score_threshold) & (df["avg_div_levdist"] < sim_div_threshold)][["idx", "src", "mt", "ref", "score", "var_probs"]].to_dict("records")[1]["mt"]
df.loc[(df["f1_bpe"]==1) & (df["score"] > score_threshold) & (df["avg_div_levdist"] < sim_div_threshold)][["idx", "src", "mt", "ref", "score", "var_probs"]].to_dict("records")[1]

{'idx': 1123516,
 'src': 'Sie stehen in verschiedenen Farben zur Verfügung, womit die beste Annäherung an Ihre eigenen Autoteppiche geschaffen wird und sie in Einklang mit dem Autointerieur stehen.',
 'mt': 'They are available in different colours, creating the best rapprochement with your own car car car car car car car car car and in harmony with the car engineer.',
 'ref': 'They are available in different colours, which enable the best possible approximation to your basic mat or congruity with the interior of your car.',
 'score': -0.59659,
 'var_probs': 0.54228496}

In [46]:
t = df.idx == 230466
t = list(t.values)
idx = [i for i, x in enumerate(t) if x]

In [43]:
import pickle
with open("/home/nunomg/mt-hallucinations/HALO/fairseq/data-bin/wmt18_de-en_heldout/checkpoint_best/mc-dropout-gens-lowcomet_w_bicleaner/mts-word-dropout_w_bicleaner.pkl", "rb") as f:
    x = pickle.load(f)

In [47]:
x[idx[0]]

['After all, some of them have been covered with plutonium, for the preparation of the nuclear weapons programme.',
 'Indeed, some of them were buried with plutonium to prepare the nuclear weapons programme.',
 'After all, some of these were plutonium, for the preparation of the nuclear-weapon programme.',
 'They were, in fact, partly buried with plutonium, for the preparation of the nuclear weapons programme.',
 'After all, they were partly buried with plutonium, for the preparation of the nuclear weapons programme.',
 'Indeed, they were partly filled with plutonium, for the preparation of the nuclear weapons programme.',
 'In fact, they were partly covered with plutonium, in the preparation of the nuclear weapons programme.',
 'After all, some of them were poured with plutonium, in order to prepare the nuclear weapons program.',
 'After all, some of them have been plutonium-based, for the preparation of the nuclear weapons programme.',
 'After all, these were partly covered with plut